Upload video - https://drive.google.com/file/d/12GVWPM8YZr5Xj9U_Zdt6oBfHO-Dtz2ow/ - to your google drive account (or just to this colab instance if you don't want persistance)

Takes 6-10 hours to train, wave mouse around in window occasionally so you don't get timed out

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Enter video path below


In [ ]:
video_path = "gdrive/MyDrive/p_sim_long_3_slow.npy"

In [ ]:
from abc import ABC, abstractmethod

class Simulation(ABC):

    @abstractmethod
    def reset():
        '''
        Reset simulation state
        '''
        pass

    @abstractmethod
    def sim_step():
        '''
        Run one step of the simulation
        '''
        pass

    @abstractmethod
    def draw():
        '''
        Render the state of the simulation
        '''
        pass

In [ ]:
#import Simulation, Particle, Walls

class VideoSimulation(Simulation):
    '''
    Batched Pre-rendered simulation
    Takes a numpy like array representing a video
    of a grid simulation and steps through them
    '''

    def __init__(self, data_arr, batch_size, model_steps_per_frame):
        self.sim_data = data_arr
        print(f'Initialized VideoSimulation with data: {data_arr.shape}')
        self.t_steps = data_arr.shape[0]
        self.grid_height = data_arr.shape[1]
        self.grid_width = data_arr.shape[2]
        self.batch_size = batch_size
        self.model_steps_per_frame = model_steps_per_frame
        self.initial_indices = np.arange(0,batch_size)*(self.t_steps-(self.t_steps%batch_size))//batch_size
        self.reset()

    def reset(self):
        self.cur_indices = np.copy(self.initial_indices)
        self.sim_steps = 0

    def sim_step(self, time_step):
        if self.sim_steps % self.model_steps_per_frame == 0:
            self.cur_indices += 1
            self.cur_indices = np.mod(self.cur_indices, self.t_steps)
        self.sim_steps += 1

    def draw(self):
        return self.sim_data[self.cur_indices]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import base64
import json
import numpy as np

class CAModel(nn.Module):

    def __init__(self, env_d, hidden_d, device):
        super(CAModel, self).__init__()
        self.env_d = env_d
        self.conv1 = nn.Conv2d(env_d*3, hidden_d, 1).to(device)
        self.conv2 = nn.Conv2d(hidden_d, env_d, 1).to(device)
        nn.init.zeros_(self.conv2.weight)
        nn.init.zeros_(self.conv2.bias)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return self.conv2(x)

    def pack_layer(self, weight, bias, outputType):
      in_ch, out_ch = weight.shape
      assert (in_ch%4==0)
      assert (out_ch%4==0)
      print(f'in_ch: {in_ch} out_ch: {out_ch}')
      print(f'bias.shape {bias.shape}')
      assert (bias.shape==(out_ch,))
      weight_scale, bias_scale = 1.0, 1.0
      if outputType == np.uint8:
        weight_scale = 2.0*np.abs(weight).max()
        bias_scale = 2.0*np.abs(bias).max()
        weight = np.round((weight/weight_scale+0.5)*255)
        bias = np.round((bias/bias_scale+0.5)*255)
      packed = np.vstack([weight, bias[None,...]])
      packed = packed.reshape(in_ch+1, out_ch//4, 4)
      packed = outputType(packed)
      packed_b64 = base64.b64encode(packed.tobytes()).decode('ascii')
      return {'data_b64': packed_b64, 'in_ch': in_ch, 'out_ch': out_ch,
              'weight_scale': weight_scale, 'bias_scale': bias_scale,
              'type': outputType.__name__}

    def export_pytorch_ca_to_webgl_demo(self, outputType=np.uint8):
      # reorder the first layer inputs to meet webgl demo perception layout
      w1 = self.conv1.weight.squeeze().cpu().detach().permute(1,0).numpy()
      w1 = w1.reshape(self.env_d, 3, -1).transpose(1, 0, 2).reshape(3*self.env_d, -1)
      print(f'w1 shape: {w1.shape}')
      w2 = self.conv2.weight.squeeze().cpu().detach().permute(1,0).numpy()

      layers = [
          self.pack_layer(w1, self.conv1.bias.cpu().detach().numpy(), outputType),
          self.pack_layer(w2, self.conv2.bias.cpu().detach().numpy(), outputType)
      ]
      return json.dumps(layers)

In [ ]:
import torch
import torch.nn.functional as F

#import Simulation

class CASimulation(Simulation):

    def __init__(self, ca_model, device, batch_size, env_size=32, env_depth=16, update_prob=0.5):
        self.model = ca_model
        self.env_size = env_size
        self.env_depth = env_depth
        self.device = device
        self.batch_size = batch_size
        self.update_probability = update_prob
        self.reset()

    def reset(self):
        self.states = torch.zeros(
            self.batch_size, self.env_depth, self.env_size, self.env_size,
            device=self.device)

    def wrap_edges(self, x):
        return F.pad(x, (1,1,1,1), 'circular', 0)

    def raw_senses(self):
        # states - (batch, depth, x, y)
        sobel_x = torch.tensor([[-1.0,0.0,1.0],[-2.0,0.0,2.0],[-1.0,0.0,1.0]], device=self.device)/8
        sobel_y = torch.tensor([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]], device=self.device)/8
        identity = torch.tensor([[0.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,0.0]], device=self.device)
        all_filters = torch.stack((identity, sobel_x, sobel_y))
        all_filters_batch = all_filters.repeat(self.env_depth,1,1).unsqueeze(1)
        return F.conv2d(
            self.wrap_edges(self.states),
            all_filters_batch,
            groups=self.env_depth
        )

    def sim_step(self, time_step):
        states_updates = self.model(self.raw_senses())*time_step
        # randomly block updates to enforce
        # asynchronous communication between cells
        rand_mask = torch.rand_like(
            self.states[:, :1, :, :], device=self.device) < self.update_probability
        self.states += states_updates*(rand_mask.float())

    def draw(self):
        return self.states[:,0:3,:,:]

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import numpy as np
from datetime import datetime
import random
import math

class CATrainer:

    def __init__(self, learned_model, ground_truth_model,
                 sim_step_blocks_per_run=4, seed=115, lr=2e-3, lr_decay=1024*1024,
                 checkpoint_interval=1, checkpoint_path='checkpoints',
                 sim_steps_per_draw=8, gt_reset_interval=512, time_step=1.0,
                 save_final_state_interval=4, save_evolution_interval=256):
        self.gt_model = ground_truth_model
        self.ml_model = learned_model
        self.gt_reset_interval = gt_reset_interval
        self.sim_step_blocks_per_run = sim_step_blocks_per_run
        self.sim_steps_per_draw = sim_steps_per_draw
        self.time_step = time_step
        self.checkpoint_interval = checkpoint_interval
        self.checkpoint_path = checkpoint_path
        self.save_final_state_interval = save_final_state_interval
        self.save_evolution_interval = save_evolution_interval
        self.run_name = "output/{:%Y_%m_%d_%H_%M_%S}".format(datetime.now())
        self.lr = lr
        self.lr_decay = lr_decay
        random.seed(seed)

    def train_standard(self, optim_steps):
        final_state_count = 0
        evolution_count = 0
        running_loss = 0
        optimizer = optim.Adam(self.ml_model.model.parameters(), lr=self.lr)
        r_losses = np.zeros(self.sim_step_blocks_per_run)
        running_loss = 0
        for o_i in tqdm(range(1, optim_steps)):

            if o_i%self.gt_reset_interval == 0:
                self.gt_model.reset()

            new_lrate = self.lr * (0.25 ** (o_i / self.lr_decay))
            for param_group in optimizer.param_groups:
                param_group['lr'] = new_lrate
            optimizer.zero_grad()

            # set initial state from ground truth model
            self.ml_model.reset()
            self.ml_model.states[:, 0:3, :, :] = self.gt_model.draw()

            c_loss = 0
            c_losses = []

            for s_i in range(self.sim_step_blocks_per_run):

                for sub_i in range(self.sim_steps_per_draw):
                    self.gt_model.sim_step(self.time_step)
                    self.ml_model.sim_step(self.time_step)
                    if o_i%self.save_evolution_interval == 0:
                        self.save_img(self.ml_model.draw()[0],
                                      'evolution_output', 'evo', evolution_count)
                        self.save_img(self.gt_model.draw()[0],
                                      'evolution_output_gt', 'evo', evolution_count)
                        evolution_count += 1

                gt_state = self.gt_model.draw()
                ml_state = self.ml_model.draw()

                loss = F.mse_loss(ml_state, gt_state)
                loss.backward(retain_graph=True)
                c_loss += loss.item()
                c_losses.append(loss.item())

            optimizer.step()

            if o_i%self.save_final_state_interval == 0:
                self.save_img(ml_state[0], 'final_state_output', 'final_state', final_state_count)
                final_state_count += 1


            if o_i == 0:
                r_losses += np.array(c_losses)
                running_loss = c_loss
            else:
                running_loss = 0.99*running_loss + 0.01*c_loss
                r_losses = 0.99*r_losses + 0.01*np.array(c_losses)
            assert math.isclose(r_losses.sum(), running_loss)
            if (o_i % 50 == 0):
                tqdm.write(f'run {o_i}, recent loss: {running_loss:.7f}, lr: {new_lrate:.5f} \nblock losses: {r_losses}')
            if o_i%self.checkpoint_interval == 0:
                self.save_model(f'ca_model_step_{o_i:06d}')

    def save_img(self, t, pth, fname, i):
        pth = self.run_name + '/' + pth
        Path(pth).mkdir(exist_ok=True, parents=True)
        normed = (torch.clamp(t.detach(),0.0,1.0)*255)
        im = Image.fromarray(normed.permute(1,2,0).cpu().numpy().astype(np.uint8))
        im.save(f'{pth}/{fname}_step_{i:06d}.png')

    def save_model(self, fname):
        pth = self.run_name + '/' + self.checkpoint_path
        Path(pth).mkdir(exist_ok=True, parents=True)
        torch.save(self.ml_model.model, f'{pth}/{fname}.pt')




In [ ]:
import torch
import numpy as np

#import VideoSimulation, CAModel, CASimulation, CATrainer


cell_dim = 16
hidden_dim = 160
batch_size = 24
train_steps = 4096*4
learning_rate = 2e-3
lr_decay_rate = 1024*24
model_steps_per_video_frame = 8
step_blocks = 16
seed = 55
video_data_path = video_path # paste in appropriate path
device = torch.device('cuda')
pretrain_path = None
if pretrain_path == None:
    ca_model = CAModel(cell_dim, hidden_dim, device)
else:
    ca_model = torch.load(pretrain_path)

video_data = torch.tensor(
    np.load(video_data_path).astype(np.float32)/255,
    device=device).permute(0, 3, 1, 2)

video_sim = VideoSimulation(
    video_data,
    batch_size,
    model_steps_per_video_frame
)

ca_sim = CASimulation(
    ca_model, device, batch_size,
    env_size=video_data.shape[2], env_depth=cell_dim
)

trainer = CATrainer(
    ca_sim, video_sim, sim_step_blocks_per_run=step_blocks,
    save_evolution_interval=1024, lr=learning_rate, lr_decay=lr_decay_rate,
    save_final_state_interval=16, sim_steps_per_draw=model_steps_per_video_frame,
    seed=seed, gt_reset_interval=10000000, checkpoint_path=f'checkpoints_c{cell_dim}_h{hidden_dim}'
)
trainer.train_standard(train_steps)


NameError: ignored